<a href="https://colab.research.google.com/github/maheravi/Deep-Learning/blob/main/FAMnist_PyTorch/FAMnist_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import FashionMNIST

In [87]:
# hyperparameters
latent_size = 10
disc_inp_sz = 28*28
img_size = 28
epochs = 10
batch_size = 32
lr = 0.001

In [94]:
class MyModel(nn.Module):
    def __init__(self, input_dims, output_dims):
        super(MyModel, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dims, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.25),
            
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.5),

            nn.Linear(128, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.5),
            
            nn.Linear(64, output_dims),
            nn.Sigmoid(),
                        )
        
    def forward(self, input_t):
      input_t = input_t.reshape((input_t.shape[0], 784))
      return self.fc(input_t)

In [89]:
 device = torch.device("cuda")
# device = torch.device("cpu")

model = MyModel(disc_inp_sz, latent_size).to(device)

model = model.to(device)
model.train(True)

MyModel(
  (fc): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.25, inplace=False)
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): ReLU()
    (6): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.5, inplace=False)
    (8): Linear(in_features=128, out_features=64, bias=True)
    (9): ReLU()
    (10): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Dropout(p=0.5, inplace=False)
    (12): Linear(in_features=64, out_features=10, bias=True)
    (13): Sigmoid()
  )
)

In [90]:
def calc_acc(preds, labels):
    _, preds_max = torch.max(preds, 1)
    acc = torch.sum(preds_max == labels.data, dtype=torch.float64) / len(preds)
    return acc

In [91]:
# Data Preparing

transform = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0), (1))
])

dataset = torchvision.datasets.FashionMNIST("./dataset", train=True, download=True, transform=transform)
train_data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [92]:
 # compile
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_function = torch.nn.CrossEntropyLoss()

In [93]:
# train

for epoch in range(epochs):
    train_loss = 0.0
    train_acc = 0.0
    for images, labels in train_data_loader:
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        # 1- forwarding
        preds = model(images)
        # 2- backwarding 
        loss = loss_function(preds, labels)
        loss.backward()
        # 3- Update
        optimizer.step()

        train_loss += loss
        train_acc += calc_acc(preds, labels)
    
    total_loss = train_loss / len(train_data_loader)
    total_acc = train_acc / len(train_data_loader)

    print(f"Epoch: {epoch}, Loss: {total_loss}, Acc: {total_acc}")

Epoch: 0, Loss: 1.7172375917434692, Acc: 0.6479833333333334
Epoch: 1, Loss: 1.6353857517242432, Acc: 0.7182333333333334
Epoch: 2, Loss: 1.613921880722046, Acc: 0.7676000000000001
Epoch: 3, Loss: 1.6077899932861328, Acc: 0.7765833333333334
Epoch: 4, Loss: 1.6020374298095703, Acc: 0.7958500000000001
Epoch: 5, Loss: 1.5956372022628784, Acc: 0.8069500000000001
Epoch: 6, Loss: 1.5943416357040405, Acc: 0.8101166666666667
Epoch: 7, Loss: 1.5956722497940063, Acc: 0.8080333333333334
Epoch: 8, Loss: 1.5932137966156006, Acc: 0.81045
Epoch: 9, Loss: 1.5891281366348267, Acc: 0.8163333333333334


In [95]:
# save
torch.save(model.state_dict(), "FAMnist.pth")

In [ ]:
# inference

import cv2
import numpy as np


# model.train(False)
model.eval()

# preprocess
img = cv2.imread("3.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = cv2.resize(img, (28, 28))
tensor = transform(img).unsqueeze(0).to(device)

# process
preds = model(tensor)

# postprocess
preds = preds.cpu().detach().numpy()
output = np.argmax(preds)
output

3